# MetadataScrape

In this notebook, I explore some possible Internet sources of metadata.

In [1]:
from bs4 import BeautifulSoup
import requests

## WorldCat

In [12]:
params = {
    'q': 'Abdul Raof Hussein On the stylistic variation in the Quranic genre',
    'qt': 'results_page'
}
response = requests.get('https://www.worldcat.org/search', params=params)
'New horizons' in response.text

True

In [20]:
soup = BeautifulSoup(response.content, 'lxml')
first_result = soup.select('tr.menuElem')[0]
first_result

<tr class="menuElem">
<td class="num"><input id="itemid_1000593129" name="itemid" type="checkbox" value="1000593129"/><label for="itemid_1000593129" style="display:none">1. New horizons in Qur'anic linguistics : a syntactic, semantics and stylistic analysis</label></td>
<td class="num">1.</td>
<td class="coverart">
<a href="/title/new-horizons-in-quranic-linguistics-a-syntactic-semantics-and-stylistic-analysis/oclc/1000593129&amp;referer=brief_results"> <img :="" a="" abdul-raof="" alt="New horizons in Qur" anic="" by="" hussein="" linguistics="" src="//coverart.oclc.org/ImageWebSvc/oclc/+-+4768405556_70.jpg?SearchOrder=+-+OT,OS,TN,GO,FA" syntactic="" title="New horizons in Qur" width="70"/></a>
</td>
<td class="result details">
<div class="oclc_number" data-source-collection="/XWC/">1000593129</div>
<div class="item_number">1</div>
<div class="name">
<a href="/title/new-horizons-in-quranic-linguistics-a-syntactic-semantics-and-stylistic-analysis/oclc/1000593129&amp;referer=brief_resul

In [29]:
first_result.select('.coverart img')[0]['src']

'//coverart.oclc.org/ImageWebSvc/oclc/+-+4768405556_70.jpg?SearchOrder=+-+OT,OS,TN,GO,FA'

In [32]:
first_result.select('.result.details .author')[0].text

'by Hussein Abdul-Raof'

In [36]:
first_result.select('.result.details .itemLanguage')[0].text

'English'

In [47]:
class WorldCatSearch:
  """Encapsulates the results of a WorldCat search."""
  WORLDCAT = 'https://www.worldcat.org'
  def __init__(self, query):
    """Makes a WorldCat query with the keywords specified in QUERY
    and saves the result in SELF.
    """
    self.results = BeautifulSoup(
        requests.get(WorldCatSearch.WORLDCAT + '/search', {
            'q': query,
            'qt': 'results_page'
        }).content,
        'lxml'
    ).select('tr.menuElem')
  def author(self):
    """Returns the author associated with the first result of the query.
    Returns None if the desired information is unavailable.
    """
    try:
      text = self.results[0].select('.result.details .author')[0].text
    except IndexError:
      return None
    if text[:3] == 'by ':
      return text[3:]
    return text
  def cover_art(self):
    """Returns a link to the cover art associated with the first result
    of the query.
    Returns None if the desired information is unavailable.
    """
    try:
      return self.results[0].select('.coverart img')[0]['src']
    except IndexError:
      return None
  def language(self):
    """Returns the language associated with the first result of the
    query. Returns None if the desired information is unavailable.
    """
    try:
      return self.results[0].select('.result.details .itemLanguage')[0].text
    except IndexError:
      return None
  def worldcat_link(self):
    """Returns a link to a WorldCat page with details about the
    queried bibliographic resource.
    """
    try:
      return WorldCatSearch.WORLDCAT + \
          self.results[0].select('.result.details .name a')[0]['href']
    except IndexError:
      return None
  def result(self):
    """Returns a summary of the results of the query."""
    return {
        attr: method()
        for attr, method in [
            ('author', self.author),
            ('language', self.language),
            ('worldcat_link', self.worldcat_link),
            ('cover_art', self.cover_art)
        ]
    }
WorldCatSearch(
    'Abdul Raof Hussein On the stylistic variation in the Quranic genre'
    ).result()

{'author': 'Hussein Abdul-Raof',
 'cover_art': '//coverart.oclc.org/ImageWebSvc/oclc/+-+4768405556_70.jpg?SearchOrder=+-+OT,OS,TN,GO,FA',
 'language': 'English',
 'worldcat_link': 'https://www.worldcat.org/title/new-horizons-in-quranic-linguistics-a-syntactic-semantics-and-stylistic-analysis/oclc/1000593129&referer=brief_results'}